In [42]:
# import libraries
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from nltk.stem.porter import *

In [27]:
# nltk.download("all")

In [ ]:
# Load the amazon review dataset
df = pd.read_csv('./data/IMDB Dataset.csv')

In [4]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [28]:
word_tokenize(text='AAA', language='english')

['AAA']

In [34]:
df = df[:1000]

In [ ]:
# create preprocess_text function
def preprocess_text(text:str):
    
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    
    tokens = word_tokenize(text.lower())

    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]

    stemmer =  PorterStemmer()
    
    stopwords_set = set(stopwords.words("english"))
    
    tokens_2 = [stemmer.stem(token) for token in filtered_tokens if token not in stopwords_set]

    # Join the tokens back into a string
    processed_text = ' '.join(tokens_2)
    return processed_text

# apply the function df


df['review_processed'] = df['review'].apply(preprocess_text)


C:\Users\papal\AppData\Local\Temp\ipykernel_19484\3987005919.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_processed'] = df['review'].apply(preprocess_text)


In [45]:
df

,review,sentiment,review_processed
0,One of the other reviewers has mentioned that ...,positive,one review mention watch 1 oz episod youll hoo...
1,A wonderful little production. <br /><br />The...,positive,wonder littl product br br film techniqu unass...
2,I thought this was a wonderful way to spend ti...,positive,thought wonder way spend time hot summer weeke...
3,Basically there's a family where a little boy ...,negative,basic there famili littl boy jake think there ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei love time money visual stun film...
...,...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,positive,noth sacr ask erni fosseliu day everybodi vide...
996,I hated it. I hate self-aware pretentious inan...,negative,hate hate selfawar pretenti inan masquerad art...
997,I usually try to be professional and construct...,negative,usual tri profession construct critic movi god...
998,If you like me is going to see this in a film ...,negative,like go see film histori class someth like sch...
